In [1]:
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
import numpy as np
from tqdm import tqdm
import corner
import slsim.Pipelines as pipelines
import slsim.Sources as sources
import slsim.Deflectors as deflectors
from slsim.lens import Lens
from numba import njit, prange
from slsim.image_simulation import lens_image, rgb_image_from_image_list
from slsim.Util.param_util import gaussian_psf
# import standard python modules
import copy
import matplotlib.patches as mpatches
from itertools import combinations

%matplotlib inline

# import lenstronomy and hierArc modules
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
from hierarc.Sampling.ParamManager.cosmo_param import CosmoParam
from hierarc.Sampling.mcmc_sampling import MCMCSampler

from hierarc.Likelihood.LensLikelihood.double_source_plane import (
    beta_double_source_plane,
    # beta2theta_e_ratio,
)


from astropy.io import fits
from astropy.table import Table

/Users/paras/Library/CloudStorage/GoogleDrive-paras.sharma@stonybrook.edu/My Drive/SL Research/codes/repos/self/slsim/slsim/Plots/lens_plots.py:5: UserWarning: If you want to simulate images with Roman filters, please install the galsim module.
Note that this module is not supported on Windows
  from slsim.roman_image_simulation import simulate_roman_image


## Generate population of sources and (potential) deflectors

In [2]:
# define a cosmology
cosmology = "FwCDM"  # Flat wCDM cosmology
# other options are: "FLCDM FwCDM", "w0waCDM", "oLCDM"
kwargs_cosmo_true = {"h0": 70, "om": 0.3, "w": -1}  # cosmological model of the forecast

# create astropy.cosmology instance of input cosmology
cosmo_param = CosmoParam(cosmology=cosmology)
cosmo_true = cosmo_param.cosmo(kwargs_cosmo_true)

# define a sky area
sky_area = Quantity(value=.5, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# Generate galaxy population using skypy pipeline.
galaxy_simulation_pipeline = pipelines.SkyPyPipeline(
    skypy_config=None, sky_area=sky_area, filters=None, cosmo=cosmo_true
)

# Initiate deflector population class.
lens_galaxies = deflectors.AllLensGalaxies(
    red_galaxy_list=galaxy_simulation_pipeline.red_galaxies,
    blue_galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light=None,
    cosmo=cosmo_true,
    sky_area=sky_area,
)

# Initiate source population class.
source_galaxies = sources.Galaxies(
    galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
    kwargs_cut=kwargs_source_cut,
    cosmo=cosmo_true,
    sky_area=sky_area,
    catalog_type="skypy",
)

# make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_galaxies,
    cosmo=cosmo_true,
    sky_area=sky_area,
)

# make lensplot class for extracting rgb images
gg_plot = LensingPlots(gg_lens_pop, num_pix=100, coadd_years=10)

print(f"LensPop initialized. Potential deflectors: {gg_lens_pop.deflector_number} ; Potential sources: {gg_lens_pop.source_number}");

/Users/paras/Library/CloudStorage/GoogleDrive-paras.sharma@stonybrook.edu/My Drive/SL Research/codes/repos/self/slsim/slsim/Deflectors/all_lens_galaxies.py:54: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  red_galaxy_list = catalog_with_angular_size_in_arcsec(
/Users/paras/Library/CloudStorage/GoogleDrive-paras.sharma@stonybrook.edu/My Drive/SL Research/codes/repos/self/slsim/slsim/Deflectors/all_lens_galaxies.py:57: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  blue_galaxy_list = catalog_with_angular_size_in_arcsec(


LensPop initialized. Potential deflectors: 175238 ; Potential sources: 236717


In [3]:
# kwargs_lens_cut = {"min_image_separation": 1, "max_image_separation": 10}
kwargs_lens_cut = {}
selected_lenses = gg_lens_pop.draw_population(
    kwargs_lens_cuts=kwargs_lens_cut,
)
print(f"Number of selected lenses: {len(selected_lenses)}")

Number of selected lenses: 135


In [7]:
selected_lenses[0].lenstronomy_kwargs()

({'lens_light_model_list': ['SERSIC_ELLIPSE'],
  'lens_model_list': ['SIE', 'SHEAR', 'CONVERGENCE'],
  'source_light_model_list': ['SERSIC_ELLIPSE']},
 {'kwargs_lens': [{'theta_E': 0.5955123395719759,
    'e1': 0.3092705382684641,
    'e2': -0.11054502342629419,
    'center_x': 0.005850682352719584,
    'center_y': -0.010643665283328153},
   {'gamma1': 0.01353435210800463,
    'gamma2': -0.015023017489974994,
    'ra_0': 0,
    'dec_0': 0},
   {'kappa': 0.008019820365178065, 'ra_0': 0, 'dec_0': 0}],
  'kwargs_source': [{'magnitude': 1,
    'R_sersic': 0.03922959107320442,
    'n_sersic': 1.0,
    'e1': -0.15853497086921006,
    'e2': -0.28329820059090194,
    'center_x': 0.05679485325721026,
    'center_y': 0.09395017411233209}],
  'kwargs_lens_light': [{'magnitude': 1,
    'R_sersic': 0.6016560690370282,
    'n_sersic': 4.0,
    'e1': 0.20720545771807206,
    'e2': -0.02947510775407299,
    'center_x': 0.005850682352719584,
    'center_y': -0.010643665283328153}],
  'kwargs_ps': None}